In [26]:
import pandas as pd

def provide_account_activity(csv_path: str, account_id: str = None):
    # Read CSV
    df = pd.read_csv(csv_path)
    return df[df['account_id'] == account_id] if account_id else None


if __name__ == "__main__":
    # Example usage
    csv_file_path = "/Users/kishorekota/git/agentic_app/llm_summary_with_finetuning/large_call_center_memos.csv"
    print(provide_account_activity(csv_file_path,account_id='e3dce91d-ea4f-4d97-9514-bdeb43dacd04'))

     Memo ID                            account_id        Date Shortcode  \
115      115  e3dce91d-ea4f-4d97-9514-bdeb43dacd04  2025-03-08  NPS ACCT   
147      147  e3dce91d-ea4f-4d97-9514-bdeb43dacd04  2024-10-21      ACCT   
251      251  e3dce91d-ea4f-4d97-9514-bdeb43dacd04  2025-01-23   PIN RST   
273      273  e3dce91d-ea4f-4d97-9514-bdeb43dacd04  2025-02-13   ECS RVS   
425      425  e3dce91d-ea4f-4d97-9514-bdeb43dacd04  2024-12-14   ECS BLK   
446      446  e3dce91d-ea4f-4d97-9514-bdeb43dacd04  2025-03-20      DISP   
521      521  e3dce91d-ea4f-4d97-9514-bdeb43dacd04  2024-12-21   ECS BLK   
543      543  e3dce91d-ea4f-4d97-9514-bdeb43dacd04  2025-01-19   PIN RST   
557      557  e3dce91d-ea4f-4d97-9514-bdeb43dacd04  2025-04-16   PND DOC   
591      591  e3dce91d-ea4f-4d97-9514-bdeb43dacd04  2025-01-02   PIN RST   
633      633  e3dce91d-ea4f-4d97-9514-bdeb43dacd04  2024-11-01   ECS BLK   
675      675  e3dce91d-ea4f-4d97-9514-bdeb43dacd04  2024-11-27   FEE WVR   
731      731

In [27]:
pip install google-cloud-aiplatform

1442.15s - pydevd: Sending message related to process being replaced timed-out after 5 seconds



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip3.12 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [39]:
import vertexai


PROJECT_ID = "ai-experimentation-428115"  # Replace with your actual project ID
REGION = "us-central1"  # Choose your desired region

vertexai.init(project=PROJECT_ID, location=REGION)

from vertexai.language_models import TextEmbeddingModel, TextEmbeddingInput

def embed_text(csv_path="/Users/kishorekota/git/agentic_app/llm_summary_with_finetuning/banking_call_center_memos.csv",
               model_name="text-embedding-005",
               task_type=None,
               title=None,
               output_dimensionality=None,
               auto_truncate=True):
    """Generates text embeddings for a list of texts using a specified model."""
    df = pd.read_csv(csv_path)
    model = TextEmbeddingModel.from_pretrained(model_name)
    inputs = [
        TextEmbeddingInput(
            text=row.get('Shortcode', '')+row.get('FullForm', '')+row.get('Description', ''),
            task_type=task_type,
            title=title
        ) for _, row in df.iterrows()
    ]
    kwargs = {}
    if output_dimensionality is not None:
        kwargs["output_dimensionality"] = output_dimensionality
    if auto_truncate is not True:
        kwargs["auto_truncate"] = auto_truncate

    embeddings = model.get_embeddings(inputs, **kwargs)
    df["embeddings"] = [embedding.values for embedding in embeddings]
    return df

embed_text(
    csv_path="/Users/kishorekota/git/agentic_app/llm_summary_with_finetuning/banking_call_center_shortcodes.csv",
    model_name="text-embedding-005",
    task_type=None,
    title=None,
    output_dimensionality=None,
    auto_truncate=True
)

,Shortcode,FullForm,Description,embeddings
0,KYC,Know Your Customer,Process of verifying customer identity,"[-3.2825846574269235e-05, 0.007965657860040665..."
1,ACCT,Account,Refers to any bank account (savings/checking),"[-0.07966338098049164, 0.044580698013305664, -..."
2,NRI,Non-Resident Indian,A customer living abroad,"[-0.053829994052648544, -0.005373343359678984,..."
3,OTP,One-Time Password,Used for 2FA during transactions,"[-0.02353052981197834, 0.0028521267231553793, ..."
4,TXN,Transaction,A banking operation like debit/credit,"[-0.06104680523276329, 0.0009331476176157594, ..."
5,CC,Credit Card,Credit card-related inquiry or product,"[0.0174559373408556, 0.012108509428799152, -0...."
6,DC,Debit Card,Debit card-related issue or service,"[-0.04398996755480766, 0.0009027213673107326, ..."
7,PIN RST,PIN Reset,Customer wants to reset their card PIN,"[-0.05566681548953056, 0.0013814364792779088, ..."
8,ADDR UPD,Address Update,Request to change registered address,"[-0.07108844816684723, 0.0003186805115547031, ..."
9,FVR CHK,Fraud Verification Check,Call made to confirm suspicious transaction,"[-0.014533139765262604, -0.016086865216493607,..."


In [30]:
!gcloud auth application-default login

1627.25s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=l2kbsX1T1VhC9CzQi7VgtHFEtG2Y9h&access_type=offline&code_challenge=75XliK1MOrtIGqvCzo_RfImfIjnASYNfShmJoqjtmZo&code_challenge_method=S256


Credentials saved to file: [/Users/kishorekota/.config/gcloud/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "ai-experimentation-428115" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning the resource.


Updates are available for some Go

In [41]:
from vertexai import rag
import vertexai, pandas as pd

PROJECT_ID = "ai-experimentation-428115"  # Replace with your actual project ID
REGION = "us-central1"  # Choose your desired regio
vertexai.init(project=PROJECT_ID, location=REGION)

# ── 2·1  Create the corpus (Vector DB is provisioned for you)
embed_cfg = rag.RagEmbeddingModelConfig(
    vertex_prediction_endpoint=rag.VertexPredictionEndpoint(
        publisher_model="publishers/google/models/text-embedding-005"   # Gemini‑era embedder
    )
)
corpus = rag.create_corpus(
    display_name="memo_glossary_corpus",
    backend_config=rag.RagVectorDbConfig(rag_embedding_model_config=embed_cfg),
)
# ── 2·2  Import the glossary file
rag.import_files(
    corpus.name,
    paths=["gs://banking_shortcodes/"],
    transformation_config=rag.TransformationConfig(       # Optional chunking
        chunking_config=rag.ChunkingConfig(chunk_size=256, chunk_overlap=64)
    ),
)


In [63]:
import re
from vertexai import rag
import pandas as pd

csv_file_path = "/Users/kishorekota/git/agentic_app/llm_summary_with_finetuning/large_call_center_memos.csv"
memo_df = pd.read_csv(csv_file_path)

memo_df = memo_df[memo_df['account_id'] == 'e3dce91d-ea4f-4d97-9514-bdeb43dacd04']

memo_df.sort_values(by='Date', ascending=False, inplace=True)
memo_text = pd.concat([memo_df['Date'], memo_df['Memo']], axis=1).to_string(index=False)

CODES = r"\b[A-Z]{3,5}\b"   # tweak to your pattern

def lookup_definitions(memo_text:str, top_k:int=1):
    # Extract unique codes from the memo
    hits = set(re.findall(CODES, memo_text))
    if not hits:
        return {}
    # Run a single RAG retrieval query per code
    defs = {}
    for code in hits:
        resp = rag.retrieval_query(
            rag_resources=[rag.RagResource(rag_corpus=corpus.name)],
            text=code,
            rag_retrieval_config=rag.RagRetrievalConfig(top_k=top_k)
        )
        print(resp)
        print(resp.contexts)
        defs[code] = resp.contexts[0].text if resp.contexts else None  # Access the `text` attribute of the first context
        # for context in resp.contexts:
        #     print(context.text)
        #     defs[code] = context.text  # Access the `text` attribute of each context
        #     break  # Only take the first match
    return defs




In [64]:
from vertexai.generative_models import GenerativeModel, Tool

# Build a retrieval *tool* so Gemini can pull extra facts on its own
retrieval_tool = Tool.from_retrieval(
    retrieval=rag.Retrieval(
        source=rag.VertexRagStore(
            rag_resources=[rag.RagResource(rag_corpus=corpus.name)],
            rag_retrieval_config=rag.RagRetrievalConfig(top_k=3)
        )
    )
)
summarizer = GenerativeModel(
    model_name="gemini-2.0-flash-001",
    tools=[retrieval_tool]
)

def summarize_call(memo_text:str):
    defs = lookup_definitions(memo_text)
    glossary_block = "\n".join(f"{k} = {v}" for k,v in defs.items())
    prompt = f"""
### Glossary
{glossary_block}

### Memo
{memo_text}

### Task
Write a plain‑English call summary for the account rep. 
Use the glossary meanings, avoid jargon, and keep it under 120 words.
"""
    return summarizer.generate_content(prompt).text

print(memo_text)
summarize_call(memo_text)

      Date                                                                                              Memo
2025-04-16  2025-04-16 | PND DOC | Rep: REP143 | Note: Collection notice acknowledged; partial payment made.
2025-03-24                                                                2025-03-24 | LMT INC | Rep: REP228
2025-03-20                        2025-03-20 | DISP | Rep: REP619 | Note: Fraud alert placed on the account.
2025-03-09                                                                2025-03-09 | FVR CHK | Rep: REP755
2025-03-08                                                               2025-03-08 | NPS ACCT | Rep: REP522
2025-02-16                                                                2025-02-16 | BAL TRF | Rep: REP420
2025-02-14                                                                2025-02-14 | BAL TRF | Rep: REP350
2025-02-13                                                                2025-02-13 | ECS RVS | Rep: REP443
2025-01-23         

'On 2025-04-16, a pending document (PND DOC) was noted by REP143; the collection notice was acknowledged, and a partial payment was made. On 2025-03-24, a limit increase (LMT INC) occurred with REP228. On 2025-03-20, REP619 noted a display (DISP) and placed a fraud alert on the account.\n'